# Project: Levels of Football

Kaleb Sailer

This is the cleaning and data frame merging file.

### Packages

In [185]:
import pandas as pd 

## High School Data Cleaning

In [187]:
hs_stats_df = pd.read_csv('hs_stats_raw.csv')

In [188]:
hs_stats_df.dtypes

team_name     object
W              int64
L              int64
ppg          float64
ppg_a        float64
ppg_diff     float64
pass_yds       int64
pass_tds       int64
rush_yds       int64
rush_tds       int64
off_tds        int64
dtype: object

In [189]:
# data engineering
hs_stats_df['pass_ypg'] = round(hs_stats_df['pass_yds']/(hs_stats_df['W'] + hs_stats_df['L']), 1)

hs_stats_df['rush_ypg'] = round(hs_stats_df['rush_yds']/(hs_stats_df['W'] + hs_stats_df['L']), 1)

hs_stats_df['win_pct'] = round(hs_stats_df['W']/(hs_stats_df['W'] + hs_stats_df['L']), 2)

hs_stats_df['ypg'] = round(hs_stats_df['pass_ypg'] + hs_stats_df['rush_ypg'], 1)

hs_stats_df['state'] = 'IA'

# team level
hs_stats_df['team_level'] = 'HS'
hs_stats_df['team_level'] = hs_stats_df['team_level'].astype('category')

In [190]:
hs_stats_df.head()

,team_name,W,L,ppg,ppg_a,ppg_diff,pass_yds,pass_tds,rush_yds,rush_tds,off_tds,pass_ypg,rush_ypg,win_pct,ypg,state,team_level
0,Southeast Polk,13,0,39.5,17.5,22.0,2653,29,2175,38,67,204.1,167.3,1.00,371.4,IA,HS
1,Dowling Catholic,9,2,32.4,11.7,20.6,1402,11,1988,31,42,127.5,180.7,0.82,308.2,IA,HS
2,Ankeny,9,4,25.6,24.0,1.6,2314,20,2222,24,44,178.0,170.9,0.69,348.9,IA,HS
3,Waukee Northwest,7,3,34.9,17.6,17.3,1491,16,1794,27,43,149.1,179.4,0.70,328.5,IA,HS
4,Ankeny Centennial,7,5,33.8,14.2,19.7,1887,21,2114,31,52,157.2,176.2,0.58,333.4,IA,HS


## College Data

In [192]:
cfb_df = pd.read_csv('cfb23.csv')

In [193]:
display(cfb_df.head())

,Unnamed: 0,Off Rank,Team,Games,Win-Loss,Off Plays,Off Yards,Off Yards/Play,Off TDs,Off Yards per Game,...,Average Time of Possession per Game,Turnover Rank,Fumbles Recovered,Opponents Intercepted,Turnovers Gain,Fumbles Lost,Interceptions Thrown_y,Turnovers Lost,Turnover Margin,Avg Turnover Margin per Game
0,0,1,LSU (SEC),13,10-3,841,7065,8.40,78,543.5,...,28:51,23,4,10,14,3,5,8,6,0.46
1,1,2,Oregon (Pac-12),14,12-2,951,7440,7.82,82,531.4,...,31:03,9,6,12,18,2,5,7,11,0.79
2,2,3,Oklahoma (Big 12),13,10-3,974,6591,6.77,66,507.0,...,29:04,23,6,20,26,11,9,20,6,0.46
3,3,4,Liberty (CUSA),14,13-1,973,6988,7.18,71,499.1,...,33:04,21,4,21,25,12,6,18,7,0.50
4,4,5,Georgia (SEC),14,13-1,958,6951,7.26,69,496.5,...,33:28,43,3,14,17,7,7,14,3,0.21


In [194]:
# dropping columns by specifying what columns to keep
cfb_df = cfb_df[['Team', 'Win-Loss', 'Off Yards per Game', 'Rush Yds', 'Pass Yards',
    'Points Per Game', 'Avg Points per Game Allowed', 'Rushing TD', 'Pass Touchdowns', 'Off TDs']]

display(cfb_df.head())

,Team,Win-Loss,Off Yards per Game,Rush Yds,Pass Yards,Points Per Game,Avg Points per Game Allowed,Rushing TD,Pass Touchdowns,Off TDs
0,LSU (SEC),10-3,543.5,2659,4406,45.5,28.0,34,44,78
1,Oregon (Pac-12),12-2,531.4,2583,4857,44.2,16.5,33,49,82
2,Oklahoma (Big 12),10-3,507.0,2368,4223,41.7,23.5,32,34,66
3,Liberty (CUSA),13-1,499.1,4106,2882,38.3,23.9,39,32,71
4,Georgia (SEC),13-1,496.5,2677,4274,40.1,15.6,40,29,69


In [195]:
# drop the reclassifying rows
cfb_df = cfb_df[cfb_df['Win-Loss'] != 'Reclassifying']

# modifying data types - converting to float for calculations later
cfb_df[['Rush Yds', 'Pass Yards', 'Points Per Game', 'Avg Points per Game Allowed']] = cfb_df[['Rush Yds', 'Pass Yards', 'Points Per Game', 'Avg Points per Game Allowed']].astype('float')

In [196]:
# modfiying columns and changing column names

# Splitting 'Team' into 'team_name' and 'conference'
cfb_df['team_name'] = cfb_df['Team'].str.extract(r'^(.*?)(?=\s\()')[0].str.strip()
cfb_df['conference'] = cfb_df['Team'].str.extract(r'\(([^()]+)\)$')[0].str.strip()

# Splitting Win-Loss into 'Wins' and 'Losses'
cfb_df['W'] = cfb_df['Win-Loss'].str.split('-', expand=True)[0].astype(int)
cfb_df['L'] = cfb_df['Win-Loss'].str.split('-', expand=True)[1].astype(int)

# Calculating Rushing Yards per Game and Passing Yards per Game
cfb_df['rush_ypg'] = round(cfb_df['Rush Yds'] / (cfb_df['W'] + cfb_df['L']), 1)
cfb_df['pass_ypg'] = round(cfb_df['Pass Yards'] / (cfb_df['W'] + cfb_df['L']), 1)

# Calculating point differential and win percentage
cfb_df['ppg_diff'] = round(cfb_df['Points Per Game'] - cfb_df['Avg Points per Game Allowed'], 1)
cfb_df['win_pct'] = round(cfb_df['W'] / (cfb_df['W'] + cfb_df['L']), 2)

cfb_df = cfb_df.rename(columns={
    'Points Per Game': 'ppg',
    'Avg Points per Game Allowed': 'ppg_a',
    'Rushing TD': 'rush_tds',
    'Pass Touchdowns': 'pass_tds',
    'Off TDs': 'off_tds',
    'Off Yards per Game': 'ypg'
})

In [197]:
# dropping unneeded columns and reordering columns
cfb_df = cfb_df[[
    'team_name', 'conference', 'W', 'L', 'ppg', 'ppg_a', 'ppg_diff', 
    'pass_tds', 'rush_tds', 'off_tds', 'pass_ypg', 'rush_ypg', 
    'win_pct', 'ypg'
]]

In [198]:
# adding level
cfb_df['team_level'] = 'College'

# correcting data types
cfb_df[['conference', 'team_level']] = cfb_df[['conference', 'team_level']].astype('category')
cfb_df[['rush_tds', 'pass_tds', 'off_tds']] = cfb_df[['rush_tds', 'pass_tds', 'off_tds']].astype('int')
cfb_df['ypg'] = cfb_df['ypg'].astype('float')

cfb_df.dtypes

team_name       object
conference    category
W                int32
L                int32
ppg            float64
ppg_a          float64
ppg_diff       float64
pass_tds         int32
rush_tds         int32
off_tds          int32
pass_ypg       float64
rush_ypg       float64
win_pct        float64
ypg            float64
team_level    category
dtype: object

## NFL Data

In [200]:
nfl_df = pd.read_csv('nfl_df_raw.csv')

In [201]:
# data engineering/adding more columns
nfl_df['win_pct'] = round(nfl_df['W']/(nfl_df['W'] + nfl_df['L']), 2)

nfl_df['conference'] = ['AFC'] * 16 + ['NFC'] * 16

nfl_df['ypg'] = round(nfl_df['pass_ypg'] + nfl_df['rush_ypg'], 1)

# Dictionary mapping each team to its state abbreviation
team_state_abbr = {
    'Buffalo Bills': 'NY', 
    'Miami Dolphins': 'FL', 
    'New York Jets': 'NJ', 
    'New England Patriots': 'MA', 
    'Baltimore Ravens': 'MD', 
    'Cleveland Browns': 'OH', 
    'Pittsburgh Steelers': 'PA', 
    'Cincinnati Bengals': 'OH', 
    'Houston Texans': 'TX', 
    'Jacksonville Jaguars': 'FL', 
    'Indianapolis Colts': 'IN', 
    'Tennessee Titans': 'TN', 
    'Kansas City Chiefs': 'MO', 
    'Las Vegas Raiders': 'NV', 
    'Denver Broncos': 'CO', 
    'Los Angeles Chargers': 'CA', 
    'Dallas Cowboys': 'TX', 
    'Philadelphia Eagles': 'PA', 
    'New York Giants': 'NJ', 
    'Washington Commanders': 'MD', 
    'Detroit Lions': 'MI', 
    'Green Bay Packers': 'WI', 
    'Minnesota Vikings': 'MN', 
    'Chicago Bears': 'IL', 
    'Tampa Bay Buccaneers': 'FL', 
    'New Orleans Saints': 'LA', 
    'Atlanta Falcons': 'GA', 
    'Carolina Panthers': 'NC', 
    'San Francisco 49ers': 'CA', 
    'Los Angeles Rams': 'CA', 
    'Seattle Seahawks': 'WA', 
    'Arizona Cardinals': 'AZ'
}

# Loop through the index and assign state abbreviation based on team name
for i in nfl_df.index:
    team_name = nfl_df.loc[i, 'team_name']
    if team_name in team_state_abbr:
        nfl_df.loc[i, 'state'] = team_state_abbr[team_name]

nfl_df['team_level'] = 'NFL'

In [202]:
# making conference and state categorical
nfl_df[['conference', 'state', 'team_level']] = nfl_df[['conference', 'state', 'team_level']].astype('category')

In [203]:
display(nfl_df)

,team_name,W,L,ppg,ppg_a,ppg_diff,pass_ypg,rush_ypg,off_tds,rush_tds,pass_tds,win_pct,conference,ypg,state,team_level
0,Buffalo Bills,11,6,26.5,18.3,8.2,244.4,130.1,51,22,29,0.65,AFC,374.5,NY,NFL
1,Miami Dolphins,11,6,29.2,23.0,6.2,265.5,135.8,57,27,30,0.65,AFC,401.3,FL,NFL
2,New York Jets,7,10,15.8,20.9,-5.1,171.7,96.9,18,7,11,0.41,AFC,268.6,NJ,NFL
3,New England Patriots,4,13,13.9,21.5,-7.6,180.5,95.7,25,9,16,0.24,AFC,276.2,MA,NFL
4,Baltimore Ravens,13,4,28.4,16.5,11.9,213.8,156.5,53,26,27,0.76,AFC,370.3,MD,NFL
5,Cleveland Browns,11,6,23.3,21.3,2.0,217.2,118.6,39,15,24,0.65,AFC,335.8,OH,NFL
6,Pittsburgh Steelers,10,7,17.9,19.1,-1.2,186.1,118.2,29,16,13,0.59,AFC,304.3,PA,NFL
7,Cincinnati Bengals,9,8,21.5,22.6,-1.1,229.1,89.8,39,12,27,0.53,AFC,318.9,OH,NFL
8,Houston Texans,10,7,22.2,20.8,1.4,245.5,96.9,37,10,27,0.59,AFC,342.4,TX,NFL
9,Jacksonville Jaguars,9,8,22.2,21.8,0.4,242.7,96.8,39,17,22,0.53,AFC,339.5,FL,NFL


## Merging of the DataFrames

In [205]:
# HS and college 
hs_cfb_df = pd.concat([hs_stats_df, cfb_df], ignore_index=True)

# writing to .csv
hs_cfb_df.to_csv('hs_cfb_df.csv', encoding='utf-8')

display(hs_cfb_df)

,team_name,W,L,ppg,ppg_a,ppg_diff,pass_yds,pass_tds,rush_yds,rush_tds,off_tds,pass_ypg,rush_ypg,win_pct,ypg,state,team_level,conference
0,Southeast Polk,13,0,39.5,17.5,22.0,2653.0,29,2175.0,38,67,204.1,167.3,1.00,371.4,IA,HS,NaN
1,Dowling Catholic,9,2,32.4,11.7,20.6,1402.0,11,1988.0,31,42,127.5,180.7,0.82,308.2,IA,HS,NaN
2,Ankeny,9,4,25.6,24.0,1.6,2314.0,20,2222.0,24,44,178.0,170.9,0.69,348.9,IA,HS,NaN
3,Waukee Northwest,7,3,34.9,17.6,17.3,1491.0,16,1794.0,27,43,149.1,179.4,0.70,328.5,IA,HS,NaN
4,Ankeny Centennial,7,5,33.8,14.2,19.7,1887.0,21,2114.0,31,52,157.2,176.2,0.58,333.4,IA,HS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Eastern Mich.,6,7,19.5,26.7,-7.2,NaN,10,NaN,18,28,152.0,110.8,0.46,262.8,NaN,College,MAC
165,Iowa,10,4,15.4,14.8,0.6,NaN,9,NaN,11,20,118.6,115.9,0.71,234.6,NaN,College,Big Ten
166,James Madison,11,2,34.1,19.5,14.6,NaN,36,NaN,17,53,284.8,134.2,0.85,418.9,NaN,College,Sun Belt
167,Jacksonville St.,9,4,30.2,21.2,9.0,NaN,12,NaN,35,47,171.0,236.7,0.69,407.7,NaN,College,CUSA


In [206]:
# college and NFL
cfb_nfl_df = pd.concat([cfb_df, nfl_df], ignore_index=True)

# writing to .csv
cfb_nfl_df.to_csv('cfb_nfl_df.csv', encoding='utf-8')

display(cfb_nfl_df)

,team_name,conference,W,L,ppg,ppg_a,ppg_diff,pass_tds,rush_tds,off_tds,pass_ypg,rush_ypg,win_pct,ypg,team_level,state
0,LSU,SEC,10,3,45.5,28.0,17.5,44,34,78,338.9,204.5,0.77,543.5,College,NaN
1,Oregon,Pac-12,12,2,44.2,16.5,27.7,49,33,82,346.9,184.5,0.86,531.4,College,NaN
2,Oklahoma,Big 12,10,3,41.7,23.5,18.2,34,32,66,324.8,182.2,0.77,507.0,College,NaN
3,Liberty,CUSA,13,1,38.3,23.9,14.4,32,39,71,205.9,293.3,0.93,499.1,College,NaN
4,Georgia,SEC,13,1,40.1,15.6,24.5,29,40,69,305.3,191.2,0.93,496.5,College,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,Carolina Panthers,NFC,2,15,13.9,24.5,-10.6,13,7,20,161.2,104.1,0.12,265.3,NFL,NC
161,San Francisco 49ers,NFC,12,5,28.9,17.5,11.4,33,27,60,257.9,140.5,0.71,398.4,NFL,CA
162,Los Angeles Rams,NFC,10,7,23.8,22.2,1.6,26,18,44,239.0,120.3,0.59,359.3,NFL,CA
163,Seattle Seahawks,NFC,9,8,21.4,23.6,-2.2,23,11,34,230.0,92.9,0.53,322.9,NFL,WA


In [207]:
# HS and NFL
hs_nfl_df = pd.concat([hs_stats_df, nfl_df], ignore_index=True)

# writing to .csv
hs_nfl_df.to_csv('hs_nfl_df.csv', encoding='utf-8')

display(hs_nfl_df)

,team_name,W,L,ppg,ppg_a,ppg_diff,pass_yds,pass_tds,rush_yds,rush_tds,off_tds,pass_ypg,rush_ypg,win_pct,ypg,state,team_level,conference
0,Southeast Polk,13,0,39.5,17.5,22.0,2653.0,29,2175.0,38,67,204.1,167.3,1.00,371.4,IA,HS,NaN
1,Dowling Catholic,9,2,32.4,11.7,20.6,1402.0,11,1988.0,31,42,127.5,180.7,0.82,308.2,IA,HS,NaN
2,Ankeny,9,4,25.6,24.0,1.6,2314.0,20,2222.0,24,44,178.0,170.9,0.69,348.9,IA,HS,NaN
3,Waukee Northwest,7,3,34.9,17.6,17.3,1491.0,16,1794.0,27,43,149.1,179.4,0.70,328.5,IA,HS,NaN
4,Ankeny Centennial,7,5,33.8,14.2,19.7,1887.0,21,2114.0,31,52,157.2,176.2,0.58,333.4,IA,HS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Carolina Panthers,2,15,13.9,24.5,-10.6,NaN,13,NaN,7,20,161.2,104.1,0.12,265.3,NC,NFL,NFC
64,San Francisco 49ers,12,5,28.9,17.5,11.4,NaN,33,NaN,27,60,257.9,140.5,0.71,398.4,CA,NFL,NFC
65,Los Angeles Rams,10,7,23.8,22.2,1.6,NaN,26,NaN,18,44,239.0,120.3,0.59,359.3,CA,NFL,NFC
66,Seattle Seahawks,9,8,21.4,23.6,-2.2,NaN,23,NaN,11,34,230.0,92.9,0.53,322.9,WA,NFL,NFC


In [208]:
# all three
all_fb_df = pd.concat([hs_cfb_df, nfl_df], ignore_index=True)

# writing to .csv
all_fb_df.to_csv('all_fb.csv', encoding='utf-8')

display(all_fb_df)

,team_name,W,L,ppg,ppg_a,ppg_diff,pass_yds,pass_tds,rush_yds,rush_tds,off_tds,pass_ypg,rush_ypg,win_pct,ypg,state,team_level,conference
0,Southeast Polk,13,0,39.5,17.5,22.0,2653.0,29,2175.0,38,67,204.1,167.3,1.00,371.4,IA,HS,NaN
1,Dowling Catholic,9,2,32.4,11.7,20.6,1402.0,11,1988.0,31,42,127.5,180.7,0.82,308.2,IA,HS,NaN
2,Ankeny,9,4,25.6,24.0,1.6,2314.0,20,2222.0,24,44,178.0,170.9,0.69,348.9,IA,HS,NaN
3,Waukee Northwest,7,3,34.9,17.6,17.3,1491.0,16,1794.0,27,43,149.1,179.4,0.70,328.5,IA,HS,NaN
4,Ankeny Centennial,7,5,33.8,14.2,19.7,1887.0,21,2114.0,31,52,157.2,176.2,0.58,333.4,IA,HS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Carolina Panthers,2,15,13.9,24.5,-10.6,NaN,13,NaN,7,20,161.2,104.1,0.12,265.3,NC,NFL,NFC
197,San Francisco 49ers,12,5,28.9,17.5,11.4,NaN,33,NaN,27,60,257.9,140.5,0.71,398.4,CA,NFL,NFC
198,Los Angeles Rams,10,7,23.8,22.2,1.6,NaN,26,NaN,18,44,239.0,120.3,0.59,359.3,CA,NFL,NFC
199,Seattle Seahawks,9,8,21.4,23.6,-2.2,NaN,23,NaN,11,34,230.0,92.9,0.53,322.9,WA,NFL,NFC
